<h1> LARN SERVER </h1>

<h3> Importing Libraries </h3>

In [ ]:
import cv2
import numpy as np
import face_recognition
import os
import itertools
import pandas as pd
import tensorflow as tf
from fer import Video, FER
import sys
import dlib
import pynormalize
from moviepy.editor import *
import librosa
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import pickle
from scipy.spatial import distance as dist
from imutils.video import FileVideoStream, VideoStream
from imutils import face_utils,resize
import time
import LengthCalculator as lc          #From LengthCalculator.py
from Emotions import Emotions as em    #From Emotions.py
import threading as th
import socket
import tqdm
from joblib import load
import xgboost

<h3> Importing Pre-Trained Models </h3>

In [ ]:
head_model = tf.keras.models.load_model('head_movement_model')
eye_model = pickle.load(open('EyeBlinkModel_saved.sav', 'rb'))
speech_model = tf.keras.models.load_model("my_model") # Speech model"
expression_model = pickle.load(open('facialexprknn.sav', 'rb'))
xgb_model_loaded = pickle.load(open("micro_xgb.pkl", "rb"))

<h3> Head Movement Analysis </h3>

In [ ]:
def head_analysis(path, res, ind):
    video_name = os.path.basename(path)
    file_name = f'hm_video_{video_name}.txt'
    f = open(file_name,'w+')
    Camera = cv2.VideoCapture(path)
    _,frame = Camera.read()

    frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    box = face_recognition.face_locations(frameRGB)          

    cx_ = (box[0][3] + box[0][1])/2
    cy_ = (box[0][3] + box[0][1])/2
    cx = cx_
    cy = cy_

    MIN_MOVE=10
    while True:
        ret,frame = Camera.read()
        if ret == True:
            frameRGB = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
            box = face_recognition.face_locations(frameRGB)          

            if ( box!= [] ):
                cx = (box[0][3] + box[0][1])/2
                cy = (box[0][0] + box[0][2])/2
                cv2.rectangle( frame ,(box[0][3],box[0][2]) , (box[0][1],box[0][0]) , (0,0,255) , 2 )

                if abs(cx-cx_) > abs(cy-cy_):
                    if cx - cx_ > MIN_MOVE:
                        f.write('LEFT\n')
                    elif cx - cx_ < -MIN_MOVE:
                        f.write('RIGHT\n')
                else:
                    if cy - cy_ > MIN_MOVE:
                        f.write('DOWN\n')
                    elif cy - cy_ < -MIN_MOVE:
                        f.write('UP\n')
            key = cv2.waitKey(30)
            cx_ = cx
            cy_ = cy
            if key == 27:
                break
        else:
            break
    f.close()
    
    col_headers =['UP', 'DOWN', 'LEFT', 'RIGHT', 'LEFT_RIGHT', 'LEFT_UP', 'LEFT_DOWN', 'RIGHT_LEFT', 'RIGHT_UP', 'RIGHT_DOWN', 'UP_LEFT', 'UP_RIGHT', 'UP_DOWN', 'DOWN_LEFT', 'DOWN_RIGHT', 'DOWN_UP']
    df_head = pd.DataFrame(columns=col_headers)
    direction_count = dict()
    for col in col_headers:
        direction_count[col] = 0

    with open(file_name) as f:
        data = f.readlines()

    line1_p = 0
    line2_p = 1

    while line2_p != len(data):
        line1 = data[line1_p].rstrip()
        line2 = data[line2_p].rstrip()
        if line1 == line2:
            direction_count[line1] += 1
        else:
            temp = line1 + '_' + line2
            direction_count[temp] += 1
        line1_p += 1
        line2_p += 1
    df_head = df_head.append(direction_count, ignore_index=True)
    
    dataset = df_head.to_numpy()
    X = dataset[:,0:16]
    X = np.asarray(X).astype('float32')
    head_pred = head_model.predict(X)
    res[ind] = (head_pred > 0.5).astype(int)[0][0]

<h3> Eye Blink Analysis </h3>

In [ ]:
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

In [ ]:
def eye_analysis(path, res, ind):
    p = 'shape_predictor_68_face_landmarks.dat'
    args = {'shape_predictor':p, 'video':path}

    EYE_AR_THRESH = 0.3
    EYE_AR_CONSEC_FRAMES = 3

    COUNTER = 0
    TOTAL = 0
    FRAME = 0

    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(args["shape_predictor"])

    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
    vs = FileVideoStream(args["video"]).start()
    fileStream = True
    time.sleep(1.0)

    while True:
        if fileStream and not vs.more():
            break
        frame = vs.read()
        if frame is None:
            break
        frame = resize(frame, width=450)
        FRAME += 1
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 0)
        for rect in rects:
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)
            leftEye = shape[lStart:lEnd]
            rightEye = shape[rStart:rEnd]
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            ear = (leftEAR + rightEAR) / 2.0
            leftEyeHull = cv2.convexHull(leftEye)
            rightEyeHull = cv2.convexHull(rightEye)
            cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
            
            if ear < EYE_AR_THRESH:
                COUNTER += 1
            else:
                if COUNTER >= EYE_AR_CONSEC_FRAMES:
                    TOTAL += 1
                COUNTER = 0

            cv2.putText(frame, "Blinks: {}".format(TOTAL), (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        key = cv2.waitKey(1) & 0xFF

        if key == ord("q"):
            break
    X = (TOTAL/FRAME)*25*60
    cv2.destroyAllWindows()
    vs.stop()
    X = np.array([X])
    X = X.reshape(1,-1)
    eye_pred = 1 if eye_model.predict(X) > 0.5 else 0
    res[ind] = eye_pred

<h3> Face Expresssion Analysis </h3>

In [ ]:
def get_scores(path):
    location_videofile = path
    face_detector = FER(mtcnn=True)
    input_video = Video(location_videofile)
    processing_data = input_video.analyze(face_detector, display=False)
    vid_df = input_video.to_pandas(processing_data)
    vid_df = input_video.get_first_face(vid_df)
    vid_df = input_video.get_emotions(vid_df)
    angry = sum(vid_df.angry)
    disgust = sum(vid_df.disgust)
    fear = sum(vid_df.fear)
    happy = sum(vid_df.happy)
    sad = sum(vid_df.sad)
    surprise = sum(vid_df.surprise)
    neutral = sum(vid_df.neutral)
    emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    emotions_values = [angry, disgust, fear, happy, sad, surprise, neutral]
    score_comparisons = pd.DataFrame(emotions, columns = ['Human Emotions'])
    score_comparisons['Emotion Value from the Video'] = emotions_values
    return score_comparisons

In [ ]:
def expression_analysis(path, res, ind):
    score = get_scores(path)
    scores = score['Emotion Value from the Video'][:5]
    scores = list(score['Emotion Value from the Video'][:5])
    facial_pred = expression_model.predict([scores])
    res[ind] =(facial_pred[0])

<h3> Speech Analysis </h3>

In [ ]:
def extract_features(path):
    # Sets the name to be the path to where the file is in my computer
    # Loads the audio file as a floating point time series and assigns the default sample rate
    # Sample rate is set to 22050 by default
    X, sample_rate = librosa.load(path, res_type='kaiser_fast')
    # Generate Mel-frequency cepstral coefficients (MFCCs) from a time series 
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    # Generates a Short-time Fourier transform (STFT) to use in the chroma_stft
    stft = np.abs(librosa.stft(X))
    # Computes a chromagram from a waveform or power spectrogram.
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    # Computes a mel-scaled spectrogram.
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    # Computes spectral contrast
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    # Computes the tonal centroid features (tonnetz)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
    sr=sample_rate).T,axis=0)
    return mfccs, chroma, mel, contrast, tonnetz

In [ ]:
def speech_analysis(path, res, ind):
    video_name = os.path.basename(path)
    audioclip = AudioFileClip(path)
    audioclip.write_audiofile(video_name+".wav")
    features = extract_features(video_name+".wav")
    features = np.concatenate(features)
    features = features.reshape(1,193)
    pred = speech_model.predict(features)
    speech_pred = np.argmax(pred, axis=1)
    res[ind] = speech_pred[0]

<h3> Facial Micro Expression Model </h3>

In [ ]:
def adjust_gamma(image, gamma=1.0):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

def draw_landmarks(image,landmarks):
    coords=[]
    for index in range(68):
        coords.append((landmarks.part(index).x,landmarks.part(index).y))
    coords=np.array(coords,np.int32)
    for elems in coords:
        cv2.circle(image,(elems[0],elems[1]), 1, (0,255,0),-1)
    return image,coords

In [ ]:
def micro_expression_analysis(path, final_res, index):
    font = cv2.FONT_HERSHEY_PLAIN
    cap = cv2.VideoCapture(path) 
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
    emot=em()
    res=[]
    while True: 
        ret, frame = cap.read()
        if ret == True:
            frame=adjust_gamma(frame,1.7)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = detector(gray)
            for face in faces:
                if len(faces)==1:
                    landmarks = predictor(gray, face)
                    frame,coords=draw_landmarks(frame,landmarks)
                    lis=emot.process_data(lc.printing(frame,coords))
                    if(lis!=None):
                        res.append(lis)
        else:
            break
    cap.release()
    result = []
    for i in res:
        for j in i:
            dic = {"Left mouth corner":0,"Right mouth corner":0,"Left eyebrow":0,"Right eyebrow":0,"Left eye open":0,"Left eye close":0,"Right eye open":0,"Right eye close":0}
            if dic.__contains__(j):
                dic[j]=1
        result.append(dic)
    df1=pd.DataFrame(result)
    x=xgb_model_loaded.predict(df1.values)
    micro_pred = np.bincount(x).argmax()
    final_res[index] = micro_pred

<h3> Combination of Model </h3>

In [ ]:
def combination(path):
    acc_scores = [0.6, 0.8, 0.6, 0.8, 0.628]
    pred_scores = [0, 0, 0, 0, 0]
    t0 = th.Thread(target=eye_analysis, args=(path, pred_scores, 0))
    t1 = th.Thread(target=speech_analysis, args=(path, pred_scores, 1))
    t2 = th.Thread(target=expression_analysis, args=(path, pred_scores, 2))
    t3 = th.Thread(target=head_analysis, args=(path, pred_scores, 3))
    t4 = th.Thread(target=micro_expression_analysis, args=(path, pred_scores, 4))
    
    t0.start()
    t1.start()
    t2.start()
    t3.start()
    t4.start()
    
    t0.join()
    t1.join()
    t2.join()
    t3.join()
    t4.join()
    
    final_pred = np.dot(np.array(acc_scores), np.array(pred_scores)) / 3.428
    if final_pred > 0.5:
        return 'Truth'
    else: 
        return 'Lie'   

<h3> Server </h3>

In [ ]:
# device's IP address
SERVER_HOST = "127.0.0.1"
SERVER_PORT = 8080

# receive 4096 bytes each time
BUFFER_SIZE = 4096
SEPARATOR = "<SEPARATOR>"

# create the server socket
# TCP socket
s = socket.socket()

# bind the socket to our local address
s.bind((SERVER_HOST, SERVER_PORT))

# enabling our server to accept connections
# 10 here is the number of unaccepted connections that
# the system will allow before refusing new connections
while True:
    s.listen(10)
    print(f"[*] Listening as {SERVER_HOST}:{SERVER_PORT}")
    
    # accept connection if there is any
    client_socket, address = s.accept() 
    
    # if below code is executed, that means the sender is connected
    print(f"[+] {address} is connected.")

    # receive the file infos
    # receive using client socket, not server socket
    received = client_socket.recv(BUFFER_SIZE).decode()
    client_addr, filename, filesize = received.split(SEPARATOR)
    
    # remove absolute path if there is
    filename = os.path.basename(filename)
    
    # convert to integer
    filesize = int(filesize)
    
    # start receiving the file from the socket
    # and writing to the file stream
    with open(filename, "wb") as f:
        while True:
            # read 4096 bytes from the socket (receive)
            bytes_read = client_socket.recv(BUFFER_SIZE)
            
            if not bytes_read:    
                # nothing is received
                # file transmitting is done
                break
            
            # write to the file the bytes we just received
            f.write(bytes_read)
    
    # close the client socket
    client_socket.close()
    
    #run the combination model and obtain the result
    result = combination(filename)
    
    #create an UDP socket to send the result to the client
    clientSock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    
    #encode the result and send it to the client
    clientSock.sendto(result.encode(), (client_addr, 8040))
    
# close the server socket
s.close()